# Information Retrieval 1#
## Assignment 2: Retrieval models [100 points] ##

In this assignment you will get familiar with basic and advanced information retrieval concepts. You will implement different information retrieval ranking models and evaluate their performance.

We provide you with a Indri index. To query the index, you'll use a Python package ([pyndri](https://github.com/cvangysel/pyndri)) that allows easy access to the underlying document statistics.

For evaluation you'll use the [TREC Eval](https://github.com/usnistgov/trec_eval) utility, provided by the National Institute of Standards and Technology of the United States. TREC Eval is the de facto standard way to compute Information Retrieval measures and is frequently referenced in scientific papers.

This is a **groups-of-three assignment**, the deadline is **Wednesday, January 31st**. Code quality, informative comments and convincing analysis of the results will be considered when grading. Submission should be done through blackboard, questions can be asked on the course [Piazza](piazza.com/university_of_amsterdam/spring2018/52041inr6y/home).

### Technicalities (must-read!) ###

The assignment directory is organized as follows:
   * `./assignment.ipynb` (this file): the description of the assignment.
   * `./index/`: the index we prepared for you.
   * `./ap_88_90/`: directory with ground-truth and evaluation sets:
      * `qrel_test`: test query relevance collection (**test set**).
      * `qrel_validation`: validation query relevance collection (**validation set**).
      * `topics_title`: semicolon-separated file with query identifiers and terms.

You will need the following software packages (tested with Python 3.5 inside [Anaconda](https://conda.io/docs/user-guide/install/index.html)):
   * Python 3.5 and Jupyter
   * Indri + Pyndri (Follow the installation instructions [here](https://github.com/nickvosk/pyndri/blob/master/README.md))
   * gensim [link](https://radimrehurek.com/gensim/install.html)
   * TREC Eval [link](https://github.com/usnistgov/trec_eval)

### TREC Eval primer ###
The TREC Eval utility can be downloaded and compiled as follows:

    git clone https://github.com/usnistgov/trec_eval.git
    cd trec_eval
    make

TREC Eval computes evaluation scores given two files: ground-truth information regarding relevant documents, named *query relevance* or *qrel*, and a ranking of documents for a set of queries, referred to as a *run*. The *qrel* will be supplied by us and should not be changed. For every retrieval model (or combinations thereof) you will generate a run of the top-1000 documents for every query. The format of the *run* file is as follows:

    $query_identifier Q0 $document_identifier $rank_of_document_for_query $query_document_similarity $run_identifier
    
where
   * `$query_identifier` is the unique identifier corresponding to a query (usually this follows a sequential numbering).
   * `Q0` is a legacy field that you can ignore.
   * `$document_identifier` corresponds to the unique identifier of a document (e.g., APXXXXXXX where AP denotes the collection and the Xs correspond to a unique numerical identifier).
   * `$rank_of_document_for_query` denotes the rank of the document for the particular query. This field is ignored by TREC Eval and is only maintained for legacy support. The ranks are computed by TREC Eval itself using the `$query_document_similarity` field (see next). However, it remains good practice to correctly compute this field.
   * `$query_document_similarity` is a score indicating the similarity between query and document where a higher score denotes greater similarity.
   * `$run_identifier` is an identifier of the run. This field is for your own convenience and has no purpose beyond bookkeeping.
   
For example, say we have two queries: `Q1` and `Q2` and we rank three documents (`DOC1`, `DOC2`, `DOC3`). For query `Q1`, we find the following similarity scores `score(Q1, DOC1) = 1.0`, `score(Q1, DOC2) = 0.5`, `score(Q1, DOC3) = 0.75`; and for `Q2`: `score(Q2, DOC1) = -0.1`, `score(Q2, DOC2) = 1.25`, `score(Q1, DOC3) = 0.0`. We can generate run using the following snippet:

In [221]:
import logging
import sys
import os

def write_run(model_name, data, out_f,
              max_objects_per_query=sys.maxsize,
              skip_sorting=False):
    """
    Write a run to an output file.
    Parameters:
        - model_name: identifier of run.
        - data: dictionary mapping topic_id to object_assesments;
            object_assesments is an iterable (list or tuple) of
            (relevance, object_id) pairs.
            The object_assesments iterable is sorted by decreasing order.
        - out_f: output file stream.
        - max_objects_per_query: cut-off for number of objects per query.
    """
    for subject_id, object_assesments in data.items():
        if not object_assesments:
            logging.warning('Received empty ranking for %s; ignoring.',
                            subject_id)

            continue

        #Probe types, to make sure everything goes alright.
        assert isinstance(object_assesments[0][0], float) or \
             isinstance(object_assesments[0][0], np.float32)
        assert isinstance(object_assesments[0][1], str) or \
            isinstance(object_assesments[0][1], bytes)

        if not skip_sorting:
            object_assesments = sorted(object_assesments, reverse=True)

        if max_objects_per_query < sys.maxsize:
            object_assesments = object_assesments[:max_objects_per_query]

        if isinstance(subject_id, bytes):
            subject_id = subject_id.decode('utf8')

        for rank, (relevance, object_id) in enumerate(object_assesments):
            if isinstance(object_id, bytes):
                object_id = object_id.decode('utf8')

            out_f.write(
                '{subject} Q0 {object} {rank} {relevance} '
                '{model_name}\n'.format(
                    subject=subject_id,
                    object=object_id,
                    rank=rank + 1,
                    relevance=relevance,
                    model_name=model_name))
            
# The following writes the run to standard output.
# In your code, you should write the runs to local
# storage in order to pass them to trec_eval.
write_run(
    model_name='example',
    data={
        'Q1': ((1.0, 'DOC1'), (0.5, 'DOC2'), (0.75, 'DOC3')),
        'Q2': ((-0.1, 'DOC1'), (1.25, 'DOC2'), (0.0, 'DOC3')),
    },
    out_f=sys.stdout,
    max_objects_per_query=1000)

Q1 Q0 DOC1 1 1.0 example
Q1 Q0 DOC3 2 0.75 example
Q1 Q0 DOC2 3 0.5 example
Q2 Q0 DOC2 1 1.25 example
Q2 Q0 DOC3 2 0.0 example
Q2 Q0 DOC1 3 -0.1 example


Now, imagine that we know that `DOC1` is relevant and `DOC3` is non-relevant for `Q1`. In addition, for `Q2` we only know of the relevance of `DOC3`. The query relevance file looks like:

    Q1 0 DOC1 1
    Q1 0 DOC3 0
    Q2 0 DOC3 1
    
We store the run and qrel in files `example.run` and `example.qrel` respectively on disk. We can now use TREC Eval to compute evaluation measures. In this example, we're only interested in Mean Average Precision and we'll only show this below for brevity. However, TREC Eval outputs much more information such as NDCG, recall, precision, etc.

    $ trec_eval -m all_trec -q example.qrel example.run | grep -E "^map\s"
    > map                   	Q1	1.0000
    > map                   	Q2	0.5000
    > map                   	all	0.7500
    
Now that we've discussed the output format of rankings and how you can compute evaluation measures from these rankings, we'll now proceed with an overview of the indexing framework you'll use.

### Pyndri primer ###
For this assignment you will use [Pyndri](https://github.com/cvangysel/pyndri) [[1](https://arxiv.org/abs/1701.00749)], a python interface for [Indri](https://www.lemurproject.org/indri.php). We have indexed the document collection and you can query the index using Pyndri. We will start by giving you some examples of what Pyndri can do:

First we read the document collection index with Pyndri:

In [202]:
import pyndri

index = pyndri.Index('index/')

The loaded index can be used to access a collection of documents in an easy manner. We'll give you some examples to get some idea of what it can do, it is up to you to figure out how to use it for the remainder of the assignment.

First let's look at the number of documents, since Pyndri indexes the documents using incremental identifiers we can simply take the lowest index and the maximum document and consider the difference:

In [192]:
print("There are %d documents in this collection." % (index.maximum_document() - index.document_base()))

There are 164597 documents in this collection.


Let's take the first document out of the collection and take a look at it:

In [193]:
example_document = index.document(index.document_base())
print(example_document)

('AP890425-0001', (1360, 192, 363, 0, 880, 0, 200, 0, 894, 412, 92160, 3, 192, 0, 363, 34, 1441, 0, 174134, 0, 200, 0, 894, 412, 2652, 0, 810, 107, 49, 4903, 420, 0, 1, 48, 35, 489, 0, 35, 687, 192, 243, 0, 249311, 1877, 0, 1651, 1174, 0, 2701, 117, 412, 0, 810, 391, 245233, 1225, 5838, 16, 0, 233156, 3496, 0, 393, 17, 0, 2435, 4819, 930, 0, 0, 200, 0, 894, 0, 22, 398, 145, 0, 3, 271, 115, 0, 1176, 2777, 292, 0, 725, 192, 0, 0, 50046, 0, 1901, 1130, 0, 192, 0, 408, 0, 243779, 0, 0, 553, 192, 0, 363, 0, 3747, 0, 0, 0, 0, 1176, 0, 1239, 0, 0, 1115, 17, 0, 0, 585, 192, 1963, 0, 0, 412, 54356, 0, 773, 0, 0, 0, 192, 0, 0, 1130, 0, 363, 0, 545, 192, 0, 1174, 1901, 1130, 0, 4, 398, 145, 39, 0, 577, 0, 355, 0, 491, 0, 6025, 0, 0, 193156, 88, 34, 437, 0, 0, 1852, 0, 828, 0, 1588, 0, 0, 0, 2615, 0, 0, 107, 49, 420, 0, 0, 190, 7, 714, 2701, 0, 237, 192, 157, 0, 412, 34, 437, 0, 0, 200, 6025, 26, 0, 0, 0, 0, 363, 0, 22, 398, 145, 0, 200, 638, 126222, 6018, 0, 880, 0, 0, 161, 0, 0, 319, 894, 2701, 

Here we see a document consists of two things, a string representing the external document identifier and an integer list representing the identifiers of words that make up the document. Pyndri uses integer representations for words or terms, thus a token_id is an integer that represents a word whereas the token is the actual text of the word/term. Every id has a unique token and vice versa with the exception of stop words: words so common that there are uninformative, all of these receive the zero id.

To see what some ids and their matching tokens we take a look at the dictionary of the index:

In [17]:
token2id, id2token, _ = index.get_dictionary()
print(list(id2token.items())[:15])

[(1, 'new'), (2, 'percent'), (3, 'two'), (4, '1'), (5, 'people'), (6, 'million'), (7, '000'), (8, 'government'), (9, 'president'), (10, 'years'), (11, 'state'), (12, '2'), (13, 'states'), (14, 'three'), (15, 'time')]


Using this dictionary we can see the tokens for the (non-stop) words in our example document:

In [18]:
print([id2token[word_id] for word_id in example_document[1] if word_id > 0])

['52', 'students', 'arrested', 'takeover', 'university', 'massachusetts', 'building', 'fifty', 'two', 'students', 'arrested', 'tuesday', 'evening', 'occupying', 'university', 'massachusetts', 'building', 'overnight', 'protest', 'defense', 'department', 'funded', 'research', 'new', 'york', 'city', 'thousands', 'city', 'college', 'students', 'got', 'unscheduled', 'holiday', 'demonstrators', 'occupied', 'campus', 'administration', 'building', 'protest', 'possible', 'tuition', 'increases', 'prompting', 'officials', 'suspend', 'classes', '60', 'police', 'riot', 'gear', 'arrived', 'university', 'massachusetts', '5', 'p', 'm', 'two', 'hours', 'later', 'bus', 'drove', 'away', '29', 'students', 'camped', 'memorial', 'hall', 'students', 'charged', 'trespassing', '23', 'students', 'arrested', 'lying', 'bus', 'prevent', 'leaving', 'police', '300', 'students', 'stood', 'building', 'chanting', 'looking', 'students', 'hall', 'arrested', '35', 'students', 'occupied', 'memorial', 'hall', '1', 'p', 'm',

The reverse can also be done, say we want to look for news about the "University of Massachusetts", the tokens of that query can be converted to ids using the reverse dictionary:

In [19]:
query_tokens = index.tokenize("University of Massachusetts")
print("Query by tokens:", query_tokens)
query_id_tokens = [token2id.get(query_token,0) for query_token in query_tokens]
print("Query by ids with stopwords:", query_id_tokens)
query_id_tokens = [word_id for word_id in query_id_tokens if word_id > 0]
print("Query by ids without stopwords:", query_id_tokens)

Query by tokens: ['university', '', 'massachusetts']
Query by ids with stopwords: [200, 0, 894]
Query by ids without stopwords: [200, 894]


Naturally we can now match the document and query in the id space, let's see how often a word from the query occurs in our example document:

In [20]:
matching_words = sum([True for word_id in example_document[1] if word_id in query_id_tokens])
print("Document %s has %d word matches with query: \"%s\"." % (example_document[0], matching_words, ' '.join(query_tokens)))
print("Document %s and query \"%s\" have a %.01f%% overlap." % (example_document[0], ' '.join(query_tokens),matching_words/float(len(example_document[1]))*100))

Document AP890425-0001 has 13 word matches with query: "university  massachusetts".
Document AP890425-0001 and query "university  massachusetts" have a 2.5% overlap.


While this is certainly not everything Pyndri can do, it should give you an idea of how to use it. Please take a look at the [examples](https://github.com/cvangysel/pyndri) as it will help you a lot with this assignment.

**CAUTION**: Avoid printing out the whole index in this Notebook as it will generate a lot of output and is likely to corrupt the Notebook.

### Parsing the query file
You can parse the query file (`ap_88_89/topics_title`) using the following snippet:

In [21]:
import collections
import io
import logging
import sys

def parse_topics(file_or_files,
                 max_topics=sys.maxsize, delimiter=';'):
    assert max_topics >= 0 or max_topics is None

    topics = collections.OrderedDict()

    if not isinstance(file_or_files, list) and \
            not isinstance(file_or_files, tuple):
        if hasattr(file_or_files, '__iter__'):
            file_or_files = list(file_or_files)
        else:
            file_or_files = [file_or_files]

    for f in file_or_files:
        assert isinstance(f, io.IOBase)

        for line in f:
            assert(isinstance(line, str))

            line = line.strip()

            if not line:
                continue

            topic_id, terms = line.split(delimiter, 1)

            if topic_id in topics and (topics[topic_id] != terms):
                    logging.error('Duplicate topic "%s" (%s vs. %s).',
                                  topic_id,
                                  topics[topic_id],
                                  terms)

            topics[topic_id] = terms

            if max_topics > 0 and len(topics) >= max_topics:
                break

    return topics

with open('./ap_88_89/topics_title', 'r') as f_topics:
    print(parse_topics([f_topics]))

OrderedDict([('51', 'Airbus Subsidies'), ('52', 'South African Sanctions'), ('53', 'Leveraged Buyouts'), ('54', 'Satellite Launch Contracts'), ('55', 'Insider Trading'), ('56', 'Prime (Lending) Rate Moves, Predictions'), ('57', 'MCI'), ('58', 'Rail Strikes'), ('59', 'Weather Related Fatalities'), ('60', 'Merit-Pay vs. Seniority'), ('61', 'Israeli Role in Iran-Contra Affair'), ('62', "Military Coups D'etat"), ('63', 'Machine Translation'), ('64', 'Hostage-Taking'), ('65', 'Information Retrieval Systems'), ('66', 'Natural Language Processing'), ('67', 'Politically Motivated Civil Disturbances'), ('68', 'Health Hazards from Fine-Diameter Fibers'), ('69', 'Attempts to Revive the SALT II Treaty'), ('70', 'Surrogate Motherhood'), ('71', 'Border Incursions'), ('72', 'Demographic Shifts in the U.S.'), ('73', 'Demographic Shifts across National Boundaries'), ('74', 'Conflicting Policy'), ('75', 'Automation'), ('76', 'U.S. Constitution - Original Intent'), ('77', 'Poaching'), ('78', 'Greenpeace'

### Task 1: Implement and compare lexical IR methods [35 points] ### 

In this task you will implement a number of lexical methods for IR using the **Pyndri** framework. Then you will evaluate these methods on the dataset we have provided using **TREC Eval**.

Use the **Pyndri** framework to get statistics of the documents (term frequency, document frequency, collection frequency; **you are not allowed to use the query functionality of Pyndri**) and implement the following scoring methods in **Python**:

- [TF-IDF](http://nlp.stanford.edu/IR-book/html/htmledition/tf-idf-weighting-1.html) and 
- [BM25](http://nlp.stanford.edu/IR-book/html/htmledition/okapi-bm25-a-non-binary-model-1.html) with k1=1.2 and b=0.75. **[5 points]**
- Language models ([survey](https://drive.google.com/file/d/0B-zklbckv9CHc0c3b245UW90NE0/view))
    - Jelinek-Mercer (explore different values of 𝛌 in the range [0.1, 0.5, 0.9]). **[5 points]**
    - Dirichlet Prior (explore different values of 𝛍 [500, 1000, 1500]). **[5 points]**
    - Absolute discounting (explore different values of 𝛅 in the range [0.1, 0.5, 0.9]). **[5 points]**
    - [Positional Language Models](http://sifaka.cs.uiuc.edu/~ylv2/pub/sigir09-plm.pdf) define a language model for each position of a document, and score a document based on the scores of its PLMs. The PLM is estimated based on propagated counts of words within a document through a proximity-based density function, which both captures proximity heuristics and achieves an effect of “soft” passage retrieval. Implement the PLM, all five kernels, but only the Best position strategy to score documents. Use 𝛔 equal to 50, and Dirichlet smoothing with 𝛍 optimized on the validation set (decide how to optimize this value yourself and motivate your decision in the report). **[10 points]**
    
Implement the above methods and report evaluation measures (on the test set) using the hyper parameter values you optimized on the validation set (also report the values of the hyper parameters). Use TREC Eval to obtain the results and report on `NDCG@10`, Mean Average Precision (`MAP@1000`), `Precision@5` and `Recall@1000`.

For the language models, create plots showing `NDCG@10` with varying values of the parameters. You can do this by chaining small scripts using shell scripting (preferred) or execute trec_eval using Python's `subprocess`.

Compute significance of the results using a [two-tailed paired Student t-test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html) **[5 points]**. Be wary of false rejection of the null hypothesis caused by the [multiple comparisons problem](https://en.wikipedia.org/wiki/Multiple_comparisons_problem). There are multiple ways to mitigate this problem and it is up to you to choose one.

Analyse the results by identifying specific queries where different methods succeed or fail and discuss possible reasons that cause these differences. This is *very important* in order to understand who the different retrieval functions behave.

**NOTE**: Don’t forget to use log computations in your calculations to avoid underflows. 

**IMPORTANT**: You should structure your code around the helper functions we provide below.

In [228]:
import time
start_time = time.time()
with open('./ap_88_89/topics_title', 'r') as f_topics:
    queries = parse_topics([f_topics])

index = pyndri.Index('index/')

num_documents = index.maximum_document() - index.document_base()

dictionary = pyndri.extract_dictionary(index)

tokenized_queries = {
    query_id: [dictionary.translate_token(token)
               for token in index.tokenize(query_string)
               if dictionary.has_token(token)]
    for query_id, query_string in queries.items()}

query_term_ids = set(
    query_term_id
    for query_term_ids in tokenized_queries.values()
    for query_term_id in query_term_ids)

print('Gathering statistics about', len(query_term_ids), 'terms.')

# inverted index creation.

document_lengths = {}
unique_terms_per_document = {}

inverted_index = collections.defaultdict(dict)
collection_frequencies = collections.defaultdict(int)

total_terms = 0

for int_doc_id in range(index.document_base(), index.maximum_document()):
    ext_doc_id, doc_token_ids = index.document(int_doc_id)

    document_bow = collections.Counter(
        token_id for token_id in doc_token_ids
        if token_id > 0)
    document_length = sum(document_bow.values())

    document_lengths[int_doc_id] = document_length
    total_terms += document_length

    unique_terms_per_document[int_doc_id] = len(document_bow)

    for query_term_id in query_term_ids:
        assert query_term_id is not None

        document_term_frequency = document_bow.get(query_term_id, 0)

        if document_term_frequency == 0:
            continue

        collection_frequencies[query_term_id] += document_term_frequency
        inverted_index[query_term_id][int_doc_id] = document_term_frequency


avg_doc_length = total_terms / num_documents
print('Inverted index creation took', time.time() - start_time, 'seconds.')

Gathering statistics about 456 terms.
Inverted index creation took 46.06136512756348 seconds.


In [321]:
def run_retrieval(model_name, score_fn):
    """
    Runs a retrieval method for all the queries and writes the TREC-friendly results in a file.
    
    :param model_name: the name of the model (a string)
    :param score_fn: the scoring function (a function - see below for an example) 
    """
    run_out_path = '{}.run'.format(model_name)

    if os.path.exists(run_out_path):
        print("File already exists")
        return

    retrieval_start_time = time.time()

    print('Retrieving using', model_name)
    data = {}
    count = 0
    # TODO: fill the data dictionary. 
    # The dictionary data should have the form: query_id --> (document_score, external_doc_id)
    if score_fn == tfidf:
        for key1 in inverted_index.keys():
            for key2 in inverted_index[key1].keys():
                external_document_id,_ = index.document(key2) #Get the external doc id for this document
                if key1 in data.keys():
                    data[key1].append(tuple([score_fn(key2,key1,collection_frequencies[key1]),str(external_document_id)]))
                else:
                    data[key1] = [tuple([score_fn(key2,key1,collection_frequencies[key1]),str(external_document_id)])]
      
    if score_fn == BM25:
        print("BM25 runs")
        for query_id, query_string in queries.items():
            print(query_id)
            word_list = query_string.split()
            for word in word_list:
                word = word.lower()
                if word in token2id.keys():
                    query_term_id = token2id[word]
                    for key1 in inverted_index.keys():
                        print(key1,query_id)
                        if key1 == query_id:
                            print(query_id,word_list,query_term_id)
                            
                            
    with open(run_out_path, 'w') as f_out:
        write_run(
            model_name=model_name,
            data=data,
            out_f=f_out,
            max_objects_per_query=1000)

In [322]:
# The queries dict is an ordered dict with id's as keys and queris as values
print(queries['52'])

#Query term ids give for every unique term in the queries an id
print(list(query_term_ids)[:5])

# Inverted index gives for every query_term_id and doc_id the term frequency of that term in the document
print(inverted_index[13][54006])

# Collection frequencies gives for a collection of documents for every query term id the number of occurences of that term
print(collection_frequencies[8])

South African Sanctions
[1, 2053, 8, 4104, 6153]
1
119771


In [323]:
import numpy as np
def tfidf(int_document_id, query_term_id, document_term_freq):
    """
    Scoring function for a document and a query term
    
    :param int_document_id: the document id
    :param query_token_id: the query term id (assuming you have split the query to tokens)
    :param document_term_freq: the document term frequency of the query term 
    """
    
    score = float(np.log(1+inverted_index[query_term_id][int_document_id]) * np.log(num_documents/document_term_freq))

    return score


# combining the two functions above: 
a = str(np.random.randint(0,100))
#run_retrieval('tfidf'+a, tfidf)

# TODO implement the rest of the retrieval functions
# print(query_term_ids)
def BM25(int_document_id, query_term_ids, document_term_freq): #here we pass a list of all query term ids instead of one query at a 
    score = 0
    k1 = 1.5
    b = 0.75
    # We ignore the sum including k3 since our queries are typically very short and therefore this part is not needed
    
    for query_term_id in query_term_ids:
        print(query_term_id)
    
    return score

run_retrieval('BM25'+a, BM25)
#for query_id, query_string in queries.items():
    #print(query_id,query_string)
# TODO implement tools to help you with the analysis of the results.

Retrieving using BM2561
BM25 runs
51
1 51
3076 51
2053 51
8 51
1033 51
11 51
13 51
1042 51
19 51
20 51
21 51
66582 51
1056 51
33 51
35 51
38 51
39 51
2088 51
45097 51
690 51
1070 51
178525 51
160909 51
2097 51
54 51
55 51
57 51
68 51
69 51
72 51
2121 51
1098 51
75 51
5197 51
78 51
2127 51
80 51
2129 51
82 51
1891 51
85 51
163926 51
65624 51
97318 51
92 51
94 51
97 51
1122 51
235195 51
237310 51
3173 51
119910 51
140391 51
2152 51
3177 51
107 51
4204 51
110 51
153960 51
114 51
118 51
119 51
218232 51
1145 51
231615 51
128 51
2177 51
2178 51
2181 51
163974 51
2183 51
1160 51
1161 51
138 51
140 51
4072 51
142 51
227473 51
254101 51
159894 51
2199 51
152 51
1178 51
156 51
5279 51
2208 51
163 51
1189 51
166 51
168 51
170 51
2563 51
180 51
2230 51
2231 51
186 51
71869 51
191 51
192 51
29891 51
196 51
3269 51
86216 51
203 51
205 51
20692 51
213 51
214 51
1240 51
219 51
36062 51
229 51
380 51
235 51
236 51
238 51
116975 51
241 51
1099 51
1268 51
382 51
2294 51
248 51
1275 51
252 51
82174 51
25

3269 52
86216 52
203 52
205 52
20692 52
213 52
214 52
1240 52
219 52
36062 52
229 52
380 52
235 52
236 52
238 52
116975 52
241 52
1099 52
1268 52
382 52
2294 52
248 52
1275 52
252 52
82174 52
255 52
258 52
2313 52
1510 52
269 52
20750 52
274 52
190739 52
276 52
277 52
1302 52
140799 52
281 52
1987 52
389 52
2336 52
71970 52
3291 52
3367 52
122153 52
1322 52
2439 52
116012 52
40238 52
303 52
4400 52
2353 52
2354 52
307 52
122164 52
121054 52
310 52
3383 52
313 52
735 52
5442 52
2371 52
325 52
26337 52
328 52
249163 52
104782 52
335 52
2387 52
341 52
5465 52
1373 52
351 52
4451 52
358 52
360 52
2409 52
154987 52
90480 52
128371 52
373 52
4104 52
237289 52
377 52
1404 52
3454 52
1339 52
5506 52
2587 52
388 52
1138 52
390 52
391 52
29064 52
74122 52
395 52
397 52
1424 52
1434 52
2460 52
3485 52
5189 52
416 52
156066 52
200099 52
420 52
128358 52
61863 52
53672 52
1449 52
3655 52
1266 52
431 52
5552 52
1459 52
436 52
2485 52
203193 52
1466 52
3516 52
447 52
448 52
451 52
74181 52
88519 52
2

91954 53
78639 53
1842 53
819 53
4916 53
1847 53
824 53
91961 53
829 53
832 53
76611 53
1861 53
4934 53
250383 53
120650 53
3895 53
846 53
5967 53
1848 53
851 53
3924 53
86670 53
862 53
2911 53
51043 53
243556 53
1893 53
881 53
1906 53
4755 53
5609 53
1912 53
100217 53
191898 53
1920 53
1856 53
52100 53
4193 53
2540 53
1931 53
212881 53
83861 53
917 53
922 53
220439 53
254885 53
5030 53
1959 53
942 53
5448 53
946 53
955 53
76733 53
1982 53
2549 53
171612 53
121675 53
151493 53
4040 53
43984 53
201208 53
3027 53
982 53
985 53
192484 53
2897 53
1000 53
5098 53
53245 53
4081 53
121844 53
3063 53
3066 53
3067 53
3069 53
64853 53
1 53
3076 53
2053 53
8 53
1033 53
11 53
13 53
1042 53
19 53
20 53
21 53
66582 53
1056 53
33 53
35 53
38 53
39 53
2088 53
45097 53
690 53
1070 53
178525 53
160909 53
2097 53
54 53
55 53
57 53
68 53
69 53
72 53
2121 53
1098 53
75 53
5197 53
78 53
2127 53
80 53
2129 53
82 53
1891 53
85 53
163926 53
65624 53
97318 53
92 53
94 53
97 53
1122 53
235195 53
237310 53
3173 5

163974 54
2183 54
1160 54
1161 54
138 54
140 54
4072 54
142 54
227473 54
254101 54
159894 54
2199 54
152 54
1178 54
156 54
5279 54
2208 54
163 54
1189 54
166 54
168 54
170 54
2563 54
180 54
2230 54
2231 54
186 54
71869 54
191 54
192 54
29891 54
196 54
3269 54
86216 54
203 54
205 54
20692 54
213 54
214 54
1240 54
219 54
36062 54
229 54
380 54
235 54
236 54
238 54
116975 54
241 54
1099 54
1268 54
382 54
2294 54
248 54
1275 54
252 54
82174 54
255 54
258 54
2313 54
1510 54
269 54
20750 54
274 54
190739 54
276 54
277 54
1302 54
140799 54
281 54
1987 54
389 54
2336 54
71970 54
3291 54
3367 54
122153 54
1322 54
2439 54
116012 54
40238 54
303 54
4400 54
2353 54
2354 54
307 54
122164 54
121054 54
310 54
3383 54
313 54
735 54
5442 54
2371 54
325 54
26337 54
328 54
249163 54
104782 54
335 54
2387 54
341 54
5465 54
1373 54
351 54
4451 54
358 54
360 54
2409 54
154987 54
90480 54
128371 54
373 54
4104 54
237289 54
377 54
1404 54
3454 54
1339 54
5506 54
2587 54
388 54
1138 54
390 54
391 54
29064 54
7

1329 56
2856 56
144171 56
812 56
91954 56
78639 56
1842 56
819 56
4916 56
1847 56
824 56
91961 56
829 56
832 56
76611 56
1861 56
4934 56
250383 56
120650 56
3895 56
846 56
5967 56
1848 56
851 56
3924 56
86670 56
862 56
2911 56
51043 56
243556 56
1893 56
881 56
1906 56
4755 56
5609 56
1912 56
100217 56
191898 56
1920 56
1856 56
52100 56
4193 56
2540 56
1931 56
212881 56
83861 56
917 56
922 56
220439 56
254885 56
5030 56
1959 56
942 56
5448 56
946 56
955 56
76733 56
1982 56
2549 56
171612 56
121675 56
151493 56
4040 56
43984 56
201208 56
3027 56
982 56
985 56
192484 56
2897 56
1000 56
5098 56
53245 56
4081 56
121844 56
3063 56
3066 56
3067 56
3069 56
64853 56
1 56
3076 56
2053 56
8 56
1033 56
11 56
13 56
1042 56
19 56
20 56
21 56
66582 56
1056 56
33 56
35 56
38 56
39 56
2088 56
45097 56
690 56
1070 56
178525 56
160909 56
2097 56
54 56
55 56
57 56
68 56
69 56
72 56
2121 56
1098 56
75 56
5197 56
78 56
2127 56
80 56
2129 56
82 56
1891 56
85 56
163926 56
65624 56
97318 56
92 56
94 56
97 56
1

...

1322 86
2439 86
116012 86
40238 86
303 86
4400 86
2353 86
2354 86
307 86
122164 86
121054 86
310 86
3383 86
313 86
735 86
5442 86
2371 86
325 86
26337 86
328 86
249163 86
104782 86
335 86
2387 86
341 86
5465 86
1373 86
351 86
4451 86
358 86
360 86
2409 86
154987 86
90480 86
128371 86
373 86
4104 86
237289 86
377 86
1404 86
3454 86
1339 86
5506 86
2587 86
388 86
1138 86
390 86
391 86
29064 86
74122 86
395 86
397 86
1424 86
1434 86
2460 86
3485 86
5189 86
416 86
156066 86
200099 86
420 86
128358 86
61863 86
53672 86
1449 86
3655 86
1266 86
431 86
5552 86
1459 86
436 86
2485 86
203193 86
1466 86
3516 86
447 86
448 86
451 86
74181 86
88519 86
2636 86
39370 86
1483 86
2509 86
2510 86
77 86
243152 86
89553 86
1490 86
84387 86
6153 86
472 86
198109 86
2527 86
483 86
484 86
485 86
3558 86
178428 86
3564 86
496 86
1525 86
48553 86
3576 86
3579 86
1533 86
5630 86
1535 86
392 86
515 86
518 86
519 86
520 86
2569 86
3594 86
523 86
5814 86
192943 86
63065 86
539 86
3614 86
2598 86
1116 86
3628 86
15

2527 87
483 87
484 87
485 87
3558 87
178428 87
3564 87
496 87
1525 87
48553 87
3576 87
3579 87
1533 87
5630 87
1535 87
392 87
515 87
518 87
519 87
520 87
2569 87
3594 87
523 87
5814 87
192943 87
63065 87
539 87
3614 87
2598 87
1116 87
3628 87
1583 87
99893 87
30473 87
2619 87
2621 87
23989 87
3649 87
4675 87
580 87
1605 87
1633 87
1608 87
233033 87
117322 87
588 87
590 87
2642 87
2643 87
238862 87
1622 87
215481 87
3672 87
601 87
213404 87
5723 87
3676 87
609 87
172647 87
111208 87
54952 87
1644 87
238866 87
625 87
3698 87
629 87
631 87
250488 87
2324 87
4735 87
1664 87
141966 87
642 87
2697 87
176407 87
654 87
96348 87
659 87
73364 87
155285 87
73379 87
2673 87
1704 87
5802 87
2731 87
684 87
1709 87
2845 87
5810 87
692 87
2846 87
694 87
2743 87
5581 87
1723 87
226762 87
1312 87
188098 87
2760 87
202871 87
719 87
2296 87
63508 87
463 87
2783 87
1760 87
237281 87
738 87
187515 87
67300 87
2791 87
28392 87
2793 87
746 87
157422 87
5872 87
1781 87
1783 87
226043 87
231164 87
228094 87
811

212881 87
83861 87
917 87
922 87
220439 87
254885 87
5030 87
1959 87
942 87
5448 87
946 87
955 87
76733 87
1982 87
2549 87
171612 87
121675 87
151493 87
4040 87
43984 87
201208 87
3027 87
982 87
985 87
192484 87
2897 87
1000 87
5098 87
53245 87
4081 87
121844 87
3063 87
3066 87
3067 87
3069 87
64853 87
88
1 88
3076 88
2053 88
8 88
1033 88
11 88
13 88
1042 88
19 88
20 88
21 88
66582 88
1056 88
33 88
35 88
38 88
39 88
2088 88
45097 88
690 88
1070 88
178525 88
160909 88
2097 88
54 88
55 88
57 88
68 88
69 88
72 88
2121 88
1098 88
75 88
5197 88
78 88
2127 88
80 88
2129 88
82 88
1891 88
85 88
163926 88
65624 88
97318 88
92 88
94 88
97 88
1122 88
235195 88
237310 88
3173 88
119910 88
140391 88
2152 88
3177 88
107 88
4204 88
110 88
153960 88
114 88
118 88
119 88
218232 88
1145 88
231615 88
128 88
2177 88
2178 88
2181 88
163974 88
2183 88
1160 88
1161 88
138 88
140 88
4072 88
142 88
227473 88
254101 88
159894 88
2199 88
152 88
1178 88
156 88
5279 88
2208 88
163 88
1189 88
166 88
168 88
170 88
2

2231 88
186 88
71869 88
191 88
192 88
29891 88
196 88
3269 88
86216 88
203 88
205 88
20692 88
213 88
214 88
1240 88
219 88
36062 88
229 88
380 88
235 88
236 88
238 88
116975 88
241 88
1099 88
1268 88
382 88
2294 88
248 88
1275 88
252 88
82174 88
255 88
258 88
2313 88
1510 88
269 88
20750 88
274 88
190739 88
276 88
277 88
1302 88
140799 88
281 88
1987 88
389 88
2336 88
71970 88
3291 88
3367 88
122153 88
1322 88
2439 88
116012 88
40238 88
303 88
4400 88
2353 88
2354 88
307 88
122164 88
121054 88
310 88
3383 88
313 88
735 88
5442 88
2371 88
325 88
26337 88
328 88
249163 88
104782 88
335 88
2387 88
341 88
5465 88
1373 88
351 88
4451 88
358 88
360 88
2409 88
154987 88
90480 88
128371 88
373 88
4104 88
237289 88
377 88
1404 88
3454 88
1339 88
5506 88
2587 88
388 88
1138 88
390 88
391 88
29064 88
74122 88
395 88
397 88
1424 88
1434 88
2460 88
3485 88
5189 88
416 88
156066 88
200099 88
420 88
128358 88
61863 88
53672 88
1449 88
3655 88
1266 88
431 88
5552 88
1459 88
436 88
2485 88
203193 88
14

KeyboardInterrupt: 

### Task 2: Latent Semantic Models (LSMs) [15 points] ###

In this task you will experiment with applying distributional semantics methods ([LSI](http://lsa3.colorado.edu/papers/JASIS.lsi.90.pdf) **[5 points]** and [LDA](https://www.cs.princeton.edu/~blei/papers/BleiNgJordan2003.pdf) **[5 points]**) for retrieval.

You do not need to implement LSI or LDA on your own. Instead, you can use [gensim](http://radimrehurek.com/gensim/index.html). An example on how to integrate Pyndri with Gensim for word2vec can be found [here](https://github.com/cvangysel/pyndri/blob/master/examples/word2vec.py). For the remaining latent vector space models, you will need to implement connector classes (such as `IndriSentences`) by yourself.

In order to use a latent semantic model for retrieval, you need to:
   * build a representation of the query **q**,
   * build a representation of the document **d**,
   * calculate the similarity between **q** and **d** (e.g., cosine similarity, KL-divergence).
     
The exact implementation here depends on the latent semantic model you are using. 
   
Each of these LSMs come with various hyperparameters to tune. Make a choice on the parameters, and explicitly mention the reasons that led you to these decisions. You can use the validation set to optimize hyper parameters you see fit; motivate your decisions. In addition, mention clearly how the query/document representations were constructed for each LSM and explain your choices.

In this experiment, you will first obtain an initial top-1000 ranking for each query using TF-IDF in **Task 1**, and then re-rank the documents using the LSMs. Use TREC Eval to obtain the results and report on `NDCG@10`, Mean Average Precision (`MAP@1000`), `Precision@5` and `Recall@1000`.

Perform significance testing **[5 points]** (similar as in Task 1) in the class of semantic matching methods.

### Task 3:  Word embeddings for ranking [20 points] (open-ended) ###

First create word embeddings on the corpus we provided using [word2vec](http://arxiv.org/abs/1411.2738) -- [gensim implementation](https://radimrehurek.com/gensim/models/word2vec.html). You should extract the indexed documents using pyndri and provide them to gensim for training a model (see example [here](https://github.com/nickvosk/pyndri/blob/master/examples/word2vec.py)).
   
This is an open-ended task. It is left up you to decide how you will combine word embeddings to derive query and document representations. Note that since we provide the implementation for training word2vec, you will be graded based on your creativity on combining word embeddings for building query and document representations.

Note: If you want to experiment with pre-trained word embeddings on a different corpus, you can use the word embeddings we provide alongside the assignment (./data/reduced_vectors_google.txt.tar.gz). These are the [google word2vec word embeddings](https://code.google.com/archive/p/word2vec/), reduced to only the words that appear in the document collection we use in this assignment.

### Task 4: Learning to rank (LTR) [15 points] (open-ended) ###

In this task you will get an introduction into learning to rank for information retrieval.

You can explore different ways for devising features for the model. Obviously, you can use the retrieval methods you implemented in Task 1, Task 2 and Task 3 as features. Think about other features you can use (e.g. query/document length). Creativity on devising new features and providing motivation for them will be taken into account when grading.

For every query, first create a document candidate set using the top-1000 documents using TF-IDF, and subsequently compute features given a query and a document. Note that the feature values of different retrieval methods are likely to be distributed differently.

You are adviced to start some pointwise learning to rank algorithm e.g. logistic regression, implemented in [scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).
Train your LTR model using 10-fold cross validation on the test set. More advanced learning to rank algorithms will be appreciated when grading.

### Task 4: Write a report [15 points; instant FAIL if not provided] ###

The report should be a PDF file created using the [sigconf ACM template](https://www.acm.org/publications/proceedings-template) and will determine a significant part of your grade.

   * It should explain what you have implemented, motivate your experiments and detail what you expect to learn from them. **[10 points]**
   * Lastly, provide a convincing analysis of your results and conclude the report accordingly. **[10 points]**
      * Do all methods perform similarly on all queries? Why?
      * Is there a single retrieval model that outperforms all other retrieval models (i.e., silver bullet)?
      * ...

**Hand in the report and your self-contained implementation source files.** Only send us the files that matter, organized in a well-documented zip/tgz file with clear instructions on how to reproduce your results. That is, we want to be able to regenerate all your results with minimal effort. You can assume that the index and ground-truth information is present in the same file structure as the one we have provided.
